In [1]:
!pip install langchain
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/accelerate.git

import bs4
from langchain import hub
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

from langchain.vectorstores import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.agents.agent_toolkits import create_retriever_tool
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

from langchain_community.embeddings import HuggingFaceEmbeddings

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 299.3/299.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.0/116.0 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 9.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_id = "nachtwindecho/mistralai-Code-Instruct-Finetune-SG1-V2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
llm = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map='auto')


tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/437 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
from langchain.document_loaders import TextLoader

loader = TextLoader("/content/drive/MyDrive/Target/data.txt", encoding = 'UTF-8')
documents = loader.load()

documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)


In [14]:
# add a retriever for RAG here
!pip install sentence-transformers
!pip install faiss-gpu

from langchain_community.embeddings import HuggingFaceEmbeddings
modelPath = "intfloat/e5-large-unsupervised"
embeddings = HuggingFaceEmbeddings(model_name = modelPath,  model_kwargs = {'device':'cuda'},encode_kwargs={'normalize_embeddings':False})

# Using faiss index
from langchain.vectorstores import FAISS
db = FAISS.from_documents(texts, embeddings)

retriever = db.as_retriever()

Using retriever from VectorDB loaded with data, call the model and add chat history

In [15]:
from langchain.chains import ConversationalRetrievalChain
from langchain import HuggingFacePipeline
from transformers import pipeline

pipe = pipeline("text-generation", model=llm, tokenizer=tokenizer, max_new_tokens=128)
runllm = HuggingFacePipeline(
    pipeline = pipe,
    model_kwargs={"temperature": 0.6, "max_length": 100}
)


qa = ConversationalRetrievalChain.from_llm(runllm, retriever)

chat_history = []

query = "who is takeshi kovacs"

result = qa({"question": query, "chat_history": chat_history})

result["answer"]



'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nKovacs and Ortega are injured in an explosion orchestrated by Kadmin, who has escaped police custody. Trepp brings Kovacs to "Ray", or Reileen Kawahara. Kawahara is a Meth mob boss with whom Kovacs has dealt in the past. He had rejected her offers of partnership, believing her to be cruel and manipulative. Kawahara orders Kovacs to end his investigation, or she will torture Sarah, who is currently in virtual storage.\n\nKovacs and Ortega begin sleeping together and form a partnership. Kovacs agrees to convince Bancroft he committed suicide. His version of the story is as follows. Bancroft contracted the Rawlings virus from a brothel. The Rawlings virus is designed to scramble cortical stacks and prevent re-sleeving, causing permanent death. To prevent it from contaminating his clones, Bancroft committed suicide. Kawaha

In [16]:
chat_history = [(query, result["answer"])]

query = "and who hired him to investigate a crime"

result = qa({"question": query, "chat_history": chat_history})

result["answer"]

'Use the following pieces of context to answer the question at the end. If you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n\nKovacs and Ortega are injured in an explosion orchestrated by Kadmin, who has escaped police custody. Trepp brings Kovacs to "Ray", or Reileen Kawahara. Kawahara is a Meth mob boss with whom Kovacs has dealt in the past. He had rejected her offers of partnership, believing her to be cruel and manipulative. Kawahara orders Kovacs to end his investigation, or she will torture Sarah, who is currently in virtual storage.\n\nKovacs and Ortega begin sleeping together and form a partnership. Kovacs agrees to convince Bancroft he committed suicide. His version of the story is as follows. Bancroft contracted the Rawlings virus from a brothel. The Rawlings virus is designed to scramble cortical stacks and prevent re-sleeving, causing permanent death. To prevent it from contaminating his clones, Bancroft committed suicide. Kawaha